# Coursara Capstone project - Villages of New York

## This study will compare the villages of New York both by population and venues.

Villages have many issues that are similar to larger cities without the resources of larger cities. 
Some villages are near major cities, others are very rural in which there may not be essential services like
hospitals or even grocery stores.

We will use foursquare to determine the types of venues available in these villages or the distance to essential services.  We will group the venues using k-means to determine the villages of similar caracteristics 
that may require similar services.  
### The audiance:
State and local governments can use the results of this study to allocate resources and develop plans to help the people and their villages.  This help could be in the form of indicating where to build rural medical clinics so everyone in the state can have access to health care.

### First download the required libraries


In [ ]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
print("BeautifulSoup installed")
import pandas as pd
import numpy as np
print('Libraries imported.')


In [ ]:
#The below url contains html tables with data about New York villages.
villageurl = "https://en.wikipedia.org/wiki/List_of_villages_in_New_York_(state)"
hospitalurl = "https://en.wikipedia.org/wiki/List_of_hospitals_in_New_York_(state)"
emergencyurl ="https://profiles.health.ny.gov/hospital/county_or_region/service:Emergency+Department"
providerurl = "https://www.health.ny.gov/regulations/hcra/provider/provhosp.htm"
citylisturl ="http://www.theus50.com/newyork/cities.php"  #just has the names of the cities


In [ ]:
# get the contents of the webpage in text format and store in a variable called data
data  = requests.get(url).text

In [ ]:
#soup = BeautifulSoup(data,"lxml")
soup = BeautifulSoup(data,"html5lib")

In [ ]:
#find all html tables in the web page
tables = soup.find_all('table') # in html table is represented by the tag <table>

The following code produces the columns for the dataframe and drops rows with unassigned boroughs

In [ ]:
tables.len()

In [ ]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Village'] = row.p.text[:3]
        cell['County'] = (row.span.text).split('(')[0]
        cell['Pop.'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
#print(table_contents)

In [ ]:
df=pd.DataFrame(table_contents)
df